In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from sklearn import preprocessing
import collections

In [2]:
look_back_period = 30
future_look_period = 3

In [3]:
data = pd.read_csv('ETH-USD.csv', names='time low high open close volume'.split()) 

In [4]:
data.set_index('time', inplace = True)

In [5]:
data.sort_index(inplace = True)

In [6]:
data = data[['close','volume']] 

In [7]:
data['future_value'] = data['close'].shift(-future_look_period)
data.dropna(inplace = True)

In [8]:
data['target'] = data.apply(lambda row : 1 if (row['future_value'] >  row['close']) else 0, axis = 1) 

In [9]:
trainSet, testSet,_ = np.split(data,[int(0.95*len(data)),len(data)])

In [10]:
def preprocess_data(data_pre):
    data_pre.drop('future_value',axis = 1)
    for col in data_pre.columns:
        if col != 'target':
            data_pre[col] = data_pre[col].pct_change()
            data_pre.dropna(inplace = True)
            data_pre[col] = preprocessing.scale(data_pre[col].values)
    data_pre.dropna(inplace = True)
    sequential_data = []
    prev_days = collections.deque(maxlen = look_back_period)
    for i in data_pre.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == look_back_period:
            sequential_data.append([np.array(prev_days),i[-1]])
    random.shuffle(sequential_data)
    buys = []
    sells= []
    for seq,target in sequential_data:
        if target == 0:
            sells.append([seq,target])
        else:
            buys.append([seq,target])
    low_value = min(len(sells),len(buys))
    buys = buys [:low_value]
    sells = sells [:low_value]
    final_sequence = buys+sells
    random.shuffle(final_sequence)
    inputs = []
    decision = []
    for seq, target in final_sequence:
        inputs.append(seq)
        decision.append(target)
    return np.array(inputs), np.array(decision)

In [11]:
trainInputs, trainTargets = preprocess_data(trainSet)
testInputs, testTargets = preprocess_data(testSet)

In [12]:
model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, activation='tanh', recurrent_activation='sigmoid',return_sequences=True,dropout=0.2,input_shape = trainInputs.shape[1:]),
        tf.keras.layers.LSTM(128, activation='tanh', recurrent_activation='sigmoid',return_sequences=True,dropout=0.2),
        tf.keras.layers.LSTM(128, activation='tanh', recurrent_activation='sigmoid',return_sequences=False,dropout=0.2),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(32,activation='relu'),
        tf.keras.layers.Dense(2,activation='softmax')
])

In [13]:
model.compile(loss='sparse_categorical_crossentropy',
              optmizer='adam',
              metrics=['accuracy']
             )

In [14]:
model.fit(trainInputs, trainTargets,validation_data=(testInputs, testTargets),epochs=5,batch_size=40)

Train on 88066 samples, validate on 4356 samples
Epoch 1/5
88066/88066 [==============================] - 380s 4ms/sample - loss: 0.4080 - accuracy: 0.8130 - val_loss: 0.2390 - val_accuracy: 0.9213
Epoch 2/5
88066/88066 [==============================] - 390s 4ms/sample - loss: 0.3534 - accuracy: 0.8475 - val_loss: 0.2016 - val_accuracy: 0.9304
Epoch 3/5
88066/88066 [==============================] - 383s 4ms/sample - loss: 0.3392 - accuracy: 0.8584 - val_loss: 0.2011 - val_accuracy: 0.9284
Epoch 4/5
88066/88066 [==============================] - 426s 5ms/sample - loss: 0.3346 - accuracy: 0.8635 - val_loss: 0.1855 - val_accuracy: 0.9527
Epoch 5/5
88066/88066 [==============================] - 453s 5ms/sample - loss: 0.3318 - accuracy: 0.8647 - val_loss: 0.2029 - val_accuracy: 0.9339


In [17]:
model.save('cryptoPrediction.model')

INFO:tensorflow:Assets written to: cryptoPrediction.model/assets
